<a href="https://colab.research.google.com/github/Twenkid/Colab-Notebooks-AI-ML-CV/blob/main/MNE_bycycle_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/bycycle-tools/bycycle

Cloning into 'bycycle'...
remote: Enumerating objects: 6975, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 6975 (delta 203), reused 210 (delta 129), pack-reused 6630 (from 3)
Receiving objects: 100% (6975/6975), 119.31 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (4110/4110), done.


In [3]:
!pip install bycycle
!pip install mne
!pip install pactools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 529.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 1.3 MB/s eta 0:00:00


In [4]:
%cd bycycle

/content/bycycle


In [5]:
%cd examples/

/content/bycycle/examples


In [6]:
ls

plot_1_theta_feature_distributions.py  plot_3_pac_feature_distributions.py
plot_2_mne_feature_distributions.py    README.txt


In [7]:
!python plot_1_theta_feature_distributions.py

In [ ]:
"""
1. Theta oscillation cycle feature distributions
================================================

Compute and compare the distributions of bycycle features for two recordings.
"""

####################################################################################################

import numpy as np
import matplotlib.pyplot as plt

from neurodsp.filt import filter_signal
from neurodsp.plts import plot_time_series

from bycycle import BycycleGroup
from bycycle.plts.features import plot_feature_hist
from bycycle.utils.download import load_bycycle_data

####################################################################################################
# Load and preprocess data
# ~~~~~~~~~~~~~~~~~~~~~~~~

####################################################################################################

# Load data
ca1_raw = load_bycycle_data('ca1.npy', folder='data')
ec3_raw = load_bycycle_data('ec3.npy', folder='data')
fs = 1250
f_theta = (4, 10)

####################################################################################################

# Apply a lowpass filter at 25Hz
fc = 25
filter_seconds = .5

ca1 = filter_signal(ca1_raw, fs, 'lowpass', fc, n_seconds=filter_seconds,
                    remove_edges=False)
ec3 = filter_signal(ec3_raw, fs, 'lowpass', fc, n_seconds=filter_seconds,
                    remove_edges=False)

####################################################################################################
# Compute cycle-by-cycle features
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

####################################################################################################

# Set parameters for defining oscillatory bursts
thresholds = {
    'amp_fraction_threshold': 0,
    'amp_consistency_threshold': .6,
    'period_consistency_threshold': .75,
    'monotonicity_threshold': .8,
    'min_n_cycles': 3
}

# Cycle-by-cycle analysis
sigs = np.array([ca1, ec3])

bg = BycycleGroup(thresholds=thresholds, center_extrema='trough', return_samples=False)
bg.fit(sigs, fs, f_theta)

df_ca1, df_ec3 = bg.df_features

# Limit analysis only to oscillatory bursts
df_ca1_cycles = df_ca1[df_ca1['is_burst']]
df_ec3_cycles = df_ec3[df_ec3['is_burst']]

####################################################################################################
#
# Plot time series for each recording
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

####################################################################################################

# Choose samples to plot
samplims = (10000, 12000)
ca1_plt = ca1_raw[samplims[0]:samplims[1]]
ec3_plt = ec3_raw[samplims[0]:samplims[1]]
times = np.arange(0, len(ca1_plt)/fs, 1/fs)

fig, axes = plt.subplots(figsize=(15, 6), nrows=2)

plot_time_series(times, ca1_plt, ax=axes[0], xlim=(0, 1.6), ylim=(-2.4, 2.4),
                 xlabel="Time (s)", ylabel="CA1 Voltage (mV)")

plot_time_series(times, ec3_plt, ax=axes[1], colors='r', xlim=(0, 1.6),
                 ylim=(-2.4, 2.4), xlabel="Time (s)", ylabel="EC3 Voltage (mV)")

####################################################################################################
#
# Plot feature distributions
# ~~~~~~~~~~~~~~~~~~~~~~~~~~

####################################################################################################

fig, axes = plt.subplots(figsize=(15, 15), nrows=2, ncols=2)

# Plot cycle amplitude
cycles_ca1 = df_ca1_cycles['volt_amp']
cycles_ec3 = df_ec3_cycles['volt_amp']

plot_feature_hist(cycles_ca1, 'volt_amp', ax=axes[0][0], xlabel='Cycle amplitude (mV)',
                  xlim=(0, 4.5), color='k', bins=np.arange(0, 8, .1))

plot_feature_hist(cycles_ec3, 'volt_amp', ax=axes[0][0], xlabel='Cycle amplitude (mV)',
                  xlim=(0, 4.5), color='r', bins=np.arange(0, 8, .1))

axes[0][0].legend(['CA1', 'EC3'], fontsize=15)

# Plot cycle period
periods_ca1 = df_ca1_cycles['period'] / fs * 1000
periods_ec3 = df_ec3_cycles['period'] / fs * 1000

plot_feature_hist(periods_ca1, 'period', ax=axes[0][1], xlabel='Cycle period (ms)',
                  xlim=(0, 250), color='k', bins=np.arange(0, 250, 5))

plot_feature_hist(periods_ec3, 'volt_amp', ax=axes[0][1], xlabel='Cycle period (ms)',
                  xlim=(0, 250), color='r', bins=np.arange(0, 250, 5))

# Plot rise/decay symmetry
plot_feature_hist(df_ca1_cycles, 'time_rdsym', ax=axes[1][0], xlim=(0, 1), color='k',
                  xlabel='Rise-decay asymmetry\n(fraction of cycle in rise period)',
                  bins=np.arange(0, 1, .02))

plot_feature_hist(df_ec3_cycles, 'time_rdsym', ax=axes[1][0], xlim=(0, 1), color='r',
                  xlabel='Rise-decay asymmetry\n(fraction of cycle in rise period)',
                  bins=np.arange(0, 1, .02))

# Plot peak/trough symmetry
plot_feature_hist(df_ca1_cycles, 'time_ptsym', ax=axes[1][1], color='k',
                  xlabel='Peak-trough asymmetry\n(fraction of cycle in peak period)',
                  bins=np.arange(0, 1, .02))

plot_feature_hist(df_ec3_cycles, 'time_ptsym', ax=axes[1][1], color='r',
                  xlabel='Peak-trough asymmetry\n(fraction of cycle in peak period)',
                  bins=np.arange(0, 1, .02))


In [ ]:
"""
2. MNE Interface Cycle Feature Distributions
============================================

Compute bycycle feature distributions using MNE objects.
"""

####################################################################################################
# Import Packages and Load Data
# -----------------------------
#
# First let's import the packages we need. This example depends on mne.

####################################################################################################

import numpy as np
import matplotlib.pyplot as plt

from mne.io import read_raw_fif
from mne.datasets import sample
from mne import pick_channels

from neurodsp.plts import plot_time_series
from bycycle import BycycleGroup
from bycycle.plts import plot_feature_hist

####################################################################################################

# Frequencies of interest: the alpha band
f_alpha = (8, 15)

# Get the data path for the MNE example data
raw_fname = str(sample.data_path()) + '/MEG/sample/sample_audvis_filt-0-40_raw.fif'

# Load the file of example MNE data
raw = read_raw_fif(raw_fname, preload=True, verbose=False)

# Select EEG channels from the dataset
raw = raw.pick_types(meg=False, eeg=True, eog=False, exclude='bads')

# Grab the sampling rate from the data
fs = raw.info['sfreq']

# filter to alpha
raw = raw.filter(l_freq=None, h_freq=20.)

# Settings for exploring example channels of data
chs = ['EEG 042', 'EEG 043', 'EEG 044']
t_start = 20000
t_stop = int(t_start + (10 * fs))

# Extract an example channels to explore
sigs, times = raw.get_data(pick_channels(raw.ch_names, chs),
                           start=t_start, stop=t_stop, return_times=True)

####################################################################################################
#
# Plot time series for each recording
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Now let's see how each signal looks in time. This looks like standard EEG
# data.
#

####################################################################################################

# Plot the signal
plot_time_series(times, [sig * 1e6 for sig in sigs], labels=chs, title='EEG Signal')

####################################################################################################
# Compute cycle-by-cycle features
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Here we use the `BycycleGroup` class to compute the cycle-by-
# cycle features of the three signals.
#

####################################################################################################

# Set parameters for defining oscillatory bursts
thresholds = {
    'amp_fraction_threshold': 0.3,
    'amp_consistency_threshold': 0.4,
    'period_consistency_threshold': 0.5,
    'monotonicity_threshold': 0.8,
    'min_n_cycles': 3
}

# Create a dictionary of cycle feature dataframes, corresponding to each channel
bg = BycycleGroup(thresholds=thresholds, center_extrema='trough')
bg.fit(sigs, fs, f_alpha, axis=0)

dfs = {ch: df for df, ch in zip(bg.df_features, chs)}

####################################################################################################
#
# Plot feature distributions
# ~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# As it turns out, none of the channels in the mne example audio and visual
# task has waveform asymmetry. These data were collected from a healthy
# person while they listened to beeps or saw gratings on a screen
# so this is not unexpected.
#

####################################################################################################

fig, axes = plt.subplots(figsize=(15, 15), nrows=2, ncols=2)

for ch, df in dfs.items():

    # Rescale amplitude and period features
    df['volt_amp'] = df['volt_amp'] * 1e6
    df['period'] = df['period'] / fs * 1000

    # Plot feature histograms
    plot_feature_hist(df, 'volt_amp', only_bursts=False, ax=axes[0][0], label=ch,
                      xlabel='Cycle amplitude (mV)', bins=np.arange(0, 40, 4))

    plot_feature_hist(df, 'period', only_bursts=False, ax=axes[0][1], label=ch,
                      xlabel='Cycle period (ms)', bins=np.arange(0, 250, 25))

    plot_feature_hist(df, 'time_rdsym', only_bursts=False, ax=axes[1][0], label=ch,
                      xlabel='Rise-decay asymmetry', bins=np.arange(0, 1, .1))

    plot_feature_hist(df, 'time_ptsym', only_bursts=False, ax=axes[1][1], label=ch,
                      xlabel='Peak-trough asymmetry', bins=np.arange(0, 1, .1))


In [10]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.8 MB/s eta 0:00:00


In [14]:
"""
3. PAC Cycle Feature Distributions
==================================

Compute bycycle features for simulated phase-amplitude coupled (PAC) data.
"""

####################################################################################################
# Import packages
# ---------------
#
# First let's import the packages we need. This example depends on the
# pactools simulator to make pac and a spurious pac function from the
# pactools spurious pac example.

####################################################################################################

import numpy as np
import matplotlib.pyplot as plt

from pactools import simulate_pac, Comodulogram
from pactools.utils.pink_noise import pink_noise

from neurodsp.plts import plot_time_series
from neurodsp.sim import sim_oscillation
from neurodsp.utils.norm import normalize_variance

from bycycle import BycycleGroup
from bycycle.plts import plot_feature_hist


####################################################################################################
# Simulate PAC data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Now we will use the imported functions to make pac data, spurious pac data
# and a sine wave with pink noise.
#

####################################################################################################

n_seconds = 20
fs = 200.  # Hz
n_points = int(n_seconds * fs)

f_beta = (14, 28)
high_fq = 80.0  # Hz; carrier frequency
low_fq = 24.0  # Hz; driver frequency
low_fq_width = 2.0  # Hz

noise_level = 0.25

# Simulate beta-gamma pac
sig_pac = simulate_pac(n_points=n_points, fs=fs, high_fq=high_fq, low_fq=low_fq,
                       low_fq_width=low_fq_width, noise_level=noise_level)

# Simulate 10 Hz spiking which couples to about 60 Hz
spikes = sim_oscillation(n_seconds, fs, 10, cycle='gaussian', std=0.005)
noise = normalize_variance(pink_noise(n_points, slope=1.), variance=.5)
sig_spurious_pac = spikes + noise

# Simulate a sine wave that is the driver frequency
sig_low_fq = sim_oscillation(n_seconds, fs, low_fq)

# Add the sine wave to pink noise to make a control, no pac signal
sig_no_pac = sig_low_fq + noise

####################################################################################################
# Check comodulogram for PAC
# ~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Now, we will use the tools from pactools to compute the comodulogram which
# is a visualization of phase amplitude coupling. The stronger the driver
# phase couples with a particular frequency, the brighter the color value.
# The pac signal has 24 - 80 Hz pac as designed, the spurious pac has 10 - 60 Hz
# spurious pac and the final signal has no pac, just background noise.

####################################################################################################

fig, axs = plt.subplots(nrows=3, figsize=(10, 12), sharex=True)
titles = ['Signal with PAC', 'Signal with Spurious PAC', 'Signal with no  PAC']
for sig, ax, title in zip((sig_pac, sig_spurious_pac, sig_no_pac), axs, titles):

    # Check PAC within only channel; high and low sig are the same
    #   Use the duprelatour driven autoregressive model to fit the data
    estimator = Comodulogram(fs=fs, low_fq_range=np.arange(1, 41), low_fq_width=2.,
                             method='duprelatour', progress_bar=False)

    # Compute the comodulogram
    estimator.fit(sig)

    # Plot the results
    estimator.plot(axs=[ax], tight_layout=False, titles=[title])

####################################################################################################
#
# Plot time series for each recording
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Now let's see how each signal looks in time. The third plot of spikes is
# added to pink noise to make the second plot which is the spurious pac.
# This is so that where the spikes occur can be noted in the spurious pac plot.
#

####################################################################################################

time = np.arange(0, n_seconds, 1/fs)

fig, axes = plt.subplots(nrows=4, figsize=(16, 12), sharex=True)
xlim = (0, 1)

# Plot PAC
plot_time_series(time, sig_pac, title=titles[0], xlabel='', colors='C0', ax=axes[0], xlim=xlim)

# Plot spurious PAC
plot_time_series(time, sig_spurious_pac, title=titles[1], xlabel='', colors='C1', ax=axes[1], xlim=xlim)

# Plot spikes
plot_time_series(time, spikes, title='Spikes', xlabel='', colors='C2', ax=axes[2], xlim=xlim)

# Plot signal with no PAC
plot_time_series(time, sig_no_pac, title=titles[2], colors='C3', ax=axes[3], xlim=xlim)

####################################################################################################
# Compute cycle-by-cycle features
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Here we use the bycycle the `BycycleGroup`` class to compute the cycle-by-
# cycle features of the three signals.
#

####################################################################################################

# Set parameters for defining oscillatory bursts
thresholds = {
    'amp_fraction_threshold': 0.3,
    'amp_consistency_threshold': 0.4,
    'period_consistency_threshold': 0.5,
    'monotonicity_threshold': 0.8,
    'min_n_cycles': 3
}

sigs = np.vstack([sig_pac, sig_spurious_pac, sig_no_pac])

bg = BycycleGroup(thresholds=thresholds, center_extrema='trough', return_samples=False)

bg.fit(sigs, fs, f_beta)

dfs = {
    'pac': bg.df_features[0],
    'spurious': bg.df_features[1],
    'no_pac': bg.df_features[2],
}

####################################################################################################
#
# Plot feature distributions
# ~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# As shown in the feature distributions, the pac signal displays some peak-
# tough asymmetry as does the spurious pac signal.
#

####################################################################################################

fig, axes = plt.subplots(figsize=(15, 15), nrows=2, ncols=2)

for idx, key in enumerate(dfs):

    # Rescale periods
    dfs[key]['period'] = dfs[key]['period'] / fs * 1000

    # Plot feature histograms
    plot_feature_hist(dfs[key], 'volt_amp', only_bursts=False, ax=axes[0][0],
                      label=titles[idx], xlabel='Cycle amplitude (mV)', )

    plot_feature_hist(dfs[key], 'period', only_bursts=False, ax=axes[0][1],
                      label=titles[idx], xlabel='Cycle period (ms)')

    plot_feature_hist(dfs[key], 'time_rdsym', only_bursts=False, ax=axes[1][0],
                      label=titles[idx], xlabel='Rise-decay asymmetry')

    plot_feature_hist(dfs[key], 'time_ptsym', only_bursts=False, ax=axes[1][1],
                      label=titles[idx], xlabel='Peak-trough asymmetry')


AttributeError: module 'bigframes_vendored.ibis.__warningregistry__' has no attribute 'clear'

In [13]:
!pip install pactools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 391.2 kB/s eta 0:00:00
